### 作業目的: 使用樹型模型進行文章分類

本次作業主利用[Amazon Review data中的All Beauty](https://nijianmo.github.io/amazon/index.html)來進行review評價分類(文章分類)

資料中將review分為1,2,3,4,5分，而在這份作業，我們將評論改分為差評價、普通評價、優良評價(1,2-->1差評、3-->2普通評價、4,5-->3優良評價)

In [1]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz
!gunzip All_Beauty.json.gz

--2020-12-13 07:01:15--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47350910 (45M) [application/octet-stream]
Saving to: ‘All_Beauty.json.gz’

All_Beauty.json.gz  100%[===================>]  45.16M  39.1MB/s    in 1.2s    

2020-12-13 07:01:16 (39.1 MB/s) - ‘All_Beauty.json.gz’ saved [47350910/47350910]



### 載入套件

In [2]:
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

### 資料前處理
文本資料較為龐大，這裡我們取前10000筆資料來進行作業練習

In [3]:
# load json data
all_reviews = []
with open('All_Beauty.json', encoding='utf-8') as f:
    for review in f:
        all_reviews.append(json.loads(review))
        
all_reviews[0]

{'asin': '0143026860',
 'overall': 1.0,
 'reviewText': 'great',
 'reviewTime': '02 19, 2015',
 'reviewerID': 'A1V6B6TNIC10QE',
 'reviewerName': 'theodore j bigham',
 'summary': 'One Star',
 'unixReviewTime': 1424304000,
 'verified': True}

In [4]:
# parse label(overall) and corpus(reviewText)
corpus, labels = [], []
for review in all_reviews[:10000]:
    if 'reviewText' not in review or 'overall' not in review:
        continue
    corpus.append(review['reviewText'])
    labels.append(review['overall'])
        
# transform labels: 1,2 --> 1 and 3 --> 2 and 4,5 --> 3
label_map = {1: 1, 2: 1, 3: 2, 4: 3, 5: 3}
labels = [label_map[int(label)] for label in labels]

In [5]:
# preprocessing data
# remove email address, punctuations, and change line symbol(\n)
pattern = r'\S*@\S*|\\n|\W'
preprocess_text = lambda x: ' '.join([w for w in re.sub(pattern, ' ', x).split() if w != ''])
corpus = [preprocess_text(text) for text in corpus]

In [6]:
# split corpus and labels into train and test
x_train, x_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.2, random_state=0)
len(x_train), len(x_test), len(y_train), len(y_test)

(7996, 1999, 7996, 1999)

In [7]:
# change corpus into vector
# you can use tf-idf or BOW here
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)

# transform training and testing corpus into vector form
x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

### 訓練與預測

In [8]:
# build classification model (decision tree, random forest, or adaboost)
# start training
decision_tree_cls = DecisionTreeClassifier(criterion='entropy', max_depth=6)
decision_tree_cls.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [9]:
# start inference
y_pred = decision_tree_cls.predict(x_test)

In [10]:
# calculate accuracy
print(f"Accuracy: {decision_tree_cls.score(x_test, y_test)}")

Accuracy: 0.9004502251125562


In [11]:
# calculate confusion matrix, precision, recall, and f1-score
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.63      0.13      0.21       134
           2       0.25      0.01      0.03        73
           3       0.91      0.99      0.95      1792

    accuracy                           0.90      1999
   macro avg       0.60      0.38      0.40      1999
weighted avg       0.86      0.90      0.86      1999

[[  17    2  115]
 [   1    1   71]
 [   9    1 1782]]


由上述資訊可以發現, 模型在好評的準確度高(precision, recall都高), 而在差評的部分表現較不理想, 在普通評價的部分大部分跟差評搞混,
同學可以試著學習到的各種方法來提升模型的表現